# Data Preparation

## Import Library

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
from tqdm.notebook import tqdm
import pythainlp
from IPython.display import display
from sklearn.utils import resample 

# Load Data

In [56]:
dataset_path = '../dataset'
df_train = pd.read_csv(dataset_path + "/train.csv")
df_test = pd.read_csv(dataset_path + "/test.csv")

In [57]:
print("Train data shape: ", df_train.shape)
print("Test data shape: ", df_test.shape)
record_train = df_train.shape[0]

Train data shape:  (362, 5)
Test data shape:  (90, 4)


In [58]:
df_train.head()

,ID,set,question,answer,score
0,0,Q2,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,granularity ควรจะมีค่าต่ำ เพราะว่าเราต้องการทร...,0.0
1,1,Q3,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,เห็นด้วย เพราะเป็นการเก็บข้อมูลจากหลาย users ...,5.0
2,2,Q2,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,granularity ควรเป็น #checkout events/ #cookies...,5.0
3,3,Q3,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,"เห็นด้วย ให้ X~Binomial(N,p) โดย p เป็นอัตราส่...",2.0
4,4,Q1,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,เห็นด้วย เนื่องจากการทดสอบ A/B Testing เป็นวิธ...,4.5


In [59]:
df_train[['set', 'question', 'answer']].describe()

,set,question,answer
count,362,362,362
unique,4,4,362
top,Q4,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,ใช้ เพราะเป็นการใช้ scientific method มาช่วยหา...
freq,91,91,1


In [60]:
df_train[['score']].describe()

,score
count,362.000000
mean,3.021409
std,1.990644
min,0.000000
25%,1.000000
50%,3.500000
75%,5.000000
max,5.000000


In [61]:
df_test.head()

,ID,set,question,answer
0,362,Q3,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,ได้ เพราะ โจทย์นี้ ตัวแปรคือค่าเฉลี่ยของอัตราก...
1,363,Q4,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,50/50 มีข้อดีก็คือสามารถเก็บผลการทดสอบได้ไวกว่...
2,364,Q2,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,1.จำนวนการเข้าชม 2. จำนวนการคลิกโฆษณา 3.เวลาที...
3,365,Q3,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,เห็นด้วย เพราะการคลิกมีผลลัพธ์ได้ 2 รูปแบบ นั่...
4,366,Q4,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,50/50 - จะใช้เวลาเก็บข้อมูลน้อยกว่า แต่มีความเ...


In [62]:
df_test[['set', 'question', 'answer']].describe()

,set,question,answer
count,90,90,90
unique,4,4,90
top,Q3,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,ได้ เพราะ โจทย์นี้ ตัวแปรคือค่าเฉลี่ยของอัตราก...
freq,23,23,1


# Reading Raw Data

## Questions

In [63]:
q = 'Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาตให้ผู้ใช้อัปโหลด แชร์ และดูวิดีโอได้ แฮมทาโร่ เป็นหัวหน้าทีมการตลาดของ Hamtube และเขาต้องการทราบว่าการย้ายตำแหน่งของโฆษณาจะช่วยเพิ่มยอดขาย (ผู้ใช้คลิกโฆษณามากขึ้น) หรือไม่ ดังนั้นเขาตัดสินใจที่จะดำเนินการทดลอง A/B testing. '
print(q)

Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาตให้ผู้ใช้อัปโหลด แชร์ และดูวิดีโอได้ แฮมทาโร่ เป็นหัวหน้าทีมการตลาดของ Hamtube และเขาต้องการทราบว่าการย้ายตำแหน่งของโฆษณาจะช่วยเพิ่มยอดขาย (ผู้ใช้คลิกโฆษณามากขึ้น) หรือไม่ ดังนั้นเขาตัดสินใจที่จะดำเนินการทดลอง A/B testing. 


In [64]:
for i in range(4):
    print(f'Q{i+1}. {df_train['question'].unique()[i][269:]}')
    print()


df_train[['set']].value_counts()

Q1. granularity ควรเป็นอะไร และความยาวของ attribution period ควรเป็นเท่าไหร่ จงอธิบายเหตุผล

Q2. แฮมทาโร่เลือกที่จะใช้ binomial distribution เพื่อแทน distribution ของการทำ A/B testing ในครั้งนี้ คุณเห็นด้วยหรือไม่เห็นด้วยกับแฮมทาโร่ เพราะเหตุใด

Q3. อธิบายเหตุผลที่แฮมทาโร่ควรใช้ A/B testing ในการทดสอบ หรือเสนอข้อโต้แย้งหากคุณไม่เห็นด้วย พร้อมกับอธิบายเหตุผล

Q4. แฮมทาโร่จะต้องเลือกว่าอยากให้สัดส่วนของ user ที่เห็นโฆษณาตำแหน่งเก่า ต่อ user ที่เห็นโฆษณาตำแหน่งใหม่เป็นเท่าไร โดยตอนนี้แฮมทาโร่กำลังลังเลระหว่างสัดส่วน 50/50 กับ สัดส่วน 80/20 จงอธิบายข้อดีข้อเสียของการเลือกสัดส่วนแต่ละแบบ และตอบว่าแบบใดที่น่าจะเหมาะสมกับปัญหานี้มากกว่า



set
Q1     91
Q4     91
Q2     90
Q3     90
Name: count, dtype: int64

In [65]:
df_test[['set']].value_counts()

set
Q2     23
Q3     23
Q1     22
Q4     22
Name: count, dtype: int64

# Clean Data

In [66]:
q = 2
sample = df_train[df_train['set'] == f'Q{q}'].sample(5)

print(df_train['question'].unique()[q-1][269:])
print()
    
for record in sample.iterrows():
    print(record[1]['score'])
    print(record[1]['answer'])
    print()

แฮมทาโร่เลือกที่จะใช้ binomial distribution เพื่อแทน distribution ของการทำ A/B testing ในครั้งนี้ คุณเห็นด้วยหรือไม่เห็นด้วยกับแฮมทาโร่ เพราะเหตุใด

0.0
granularity ตามชั่วโมงหรือวัน เพื่อจับความแปรปรวนในระหว่างวันหรือสัปดาห์
attribution period ตามลักษณะสินค้า เช่น ไม่กี่วันหรือ 1 week สำหรับสินค้าที่ตัดสินใจซื้อได้เร็ว
และนานขึ้นไปสำหรับสินค้าที่ต้องใช้เวลาในการตัดสินใจมากขึ้น

5.0
ตัวเศษ คือ จำนวนครั้งการคลิกโฆษณา (เพราะต้องการวัดผลว่าจำนวนคลิกโฆษณาเพิ่มขึ้นหรือไม่)
ตัวส่วน คือ จำนวนรีเฟรชหน้าหลัก(รวมถึงตอนเข้าแอปมาครั้งแรก) (เพราะทุกๆการรีเฟรช หน้าหลักจะเลื่อนขึ้นไปบนสุดทำให้เห็นโฆษณา)
ความยาว ควรจะไม่นานเกินไป เพราะ คนที่ใช้ Hamtube ไม่ได้มีจุดประสงค์ในการดูโฆษณาเป็นหลัก โดยเฉลี่ยผู้ใช้ Hamtube จะใช้เฉลี่ยประมาณ 1 ชั่วโมงต่อวัน(มั่วเอา) จึงควรเก็บทุกๆ 1 ชั่วโมง



0.5
granularity ควรเป็นจำนวนครั้งที่ user แต่ละคนเห็นโฆษณา
attribution period เป็น 1 วัน เพื่อวัดว่าใน 1 วัน user ได้กด ads ในตำแหน่งนั้น ๆ หรือไม่

0.0
เป็นจำนวนครั้งการ show ของ ads ส่วนความยาว attribution period ใช้เป็

In [67]:
stopwords = list(pythainlp.corpus.common.thai_stopwords())
print(stopwords)

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stopwords])

['กระทั่ง', 'เสียด้วย', 'รวด', 'ส่วนมาก', 'เช่นดัง', 'เป็นๆ', 'กว้างขวาง', 'มากกว่า', 'ถูกๆ', 'อันได้แก่', 'หลังจาก', 'พวกแก', 'ถึง', 'เป็น', 'ตามที่', 'ไม่ค่อย', 'หน่อย', 'อันที่จะ', 'ได้มา', 'ก็แค่', 'ตรงๆ', 'กัน', 'จวน', 'ทั้งที', 'เราๆ', 'เป็นอันๆ', 'ส่วนดี', 'ครั้งๆ', 'เท่า', 'ยิ่งจน', 'หมด', 'คราที่', 'ถึงบัดนั้น', 'ด้วยเหตุว่า', 'พวก', 'เรื่อย', 'มิได้', 'อันใด', 'ภายภาค', 'พา', 'ค่ะ', 'กระผม', 'เชื่อ', 'ค่อนข้าง', 'ครั้งหนึ่ง', 'ดัง', 'เหลือเกิน', 'จากนี้ไป', 'เสียนี่กระไร', 'ตลอดกาล', 'ทั้งนี้', 'นับแต่นี้', 'ฉะนี้', 'พอกัน', 'ตลอดถึง', 'ทุกชิ้น', 'อย่าง', 'ก่อนหน้า', 'มักจะ', 'ประกอบ', 'ยอม', 'เสียจนกระทั่ง', 'วันใด', 'วัน', 'ขณะเดียวกัน', 'คราวนั้น', 'บางที่', 'คราวใด', 'ปัจจุบัน', 'ด้วยเหมือนกัน', 'เกิด', 'ประการ', 'คือ', 'ขอ', 'ครบ', 'พวกที่', 'หาใช่', 'เพิ่งจะ', 'ดั่งกับ', 'น่าจะ', 'พอ', 'ความ', 'ทุกอัน', 'ยิ่งใหญ่', 'ฯ', 'สุดๆ', 'ดั่งเก่า', 'แต่ทว่า', 'จากนั้น', 'ให้ไป', 'เถิด', 'หากแม้', 'ผู้', 'เช่นไร', 'ภายใต้', 'นี้แหล่', 'ที่แท้', 'เหตุผล', 'สุด', 'นับแต่นั้น', 'ทรง

In [68]:
def clean_text(text):
    text = text.str.lower()
    text = text.str.strip() # remove leading/trailing whitespaces
    # text = text.str.replace(r'http\S+', 'u', regex=True) # replace URLs with 'u'
    text = text.str.replace("\n", "", regex=False) # remove newline characters
    text = text.str.replace(r'\b\d+\.\s*', '', regex=True) # remove index numbers
    text = text.str.replace(r'[^\w\s\u0E01-\u0E5B]', ' ', regex=True) # remove all non-word characters
    # text = text.str.replace(r'\s+\d+\s+', ' n ', regex=True) # replace all numbers with 'n'
    # text = text.str.replace(r'\d+',"", regex=True) # remove all numbers
    text = text.str.replace(r'\s+', ' ', regex=True) # remove multiple spaces
    text = text.apply(remove_stopwords) # remove unimportant words
    return text

In [69]:
test = "1. ทดสอบการลบตัวเลข (50/50) ออกจากข้อความ"
print(test)
print(clean_text(pd.Series([test]))[0])

1. ทดสอบการลบตัวเลข (50/50) ออกจากข้อความ
ทดสอบการลบตัวเลข 50 50 ออกจากข้อความ


In [70]:
df_train['answer'] = clean_text(df_train['answer'])
df_test['answer'] = clean_text(df_test['answer'])

# Separate Data

In [71]:
df_train_dq = df_train.drop(columns=['question'])
df_train_dq.head()

,ID,set,answer,score
0,0,Q2,granularity ควรจะมีค่าต่ำ เพราะว่าเราต้องการทร...,0.0
1,1,Q3,เห็นด้วย เพราะเป็นการเก็บข้อมูลจากหลาย users แ...,5.0
2,2,Q2,granularity ควรเป็น checkout events cookies เพ...,5.0
3,3,Q3,เห็นด้วย x binomial n p p เป็นอัตราส่วนที่ use...,2.0
4,4,Q1,เห็นด้วย เนื่องจากการทดสอบ a b testing เป็นวิธ...,4.5


In [72]:
df_q1 = df_train_dq[df_train_dq['set'] == "Q1"]
df_q2 = df_train_dq[df_train_dq['set'] == "Q2"]
df_q3 = df_train_dq[df_train_dq['set'] == "Q3"]
df_q4 = df_train_dq[df_train_dq['set'] == "Q4"]

display(df_q1.head())

,ID,set,answer,score
4,4,Q1,เห็นด้วย เนื่องจากการทดสอบ a b testing เป็นวิธ...,4.5
5,5,Q1,แฮมทาโร่ควรใช้ a b testing ในการทดสอบ a b test...,1.0
8,8,Q1,a b testing เป็นการเปรียบเทียบผลที่เกิดขึ้นจาก...,1.0
11,11,Q1,a b testing มีประโยชน์ต่อแฮมทาโร่ในการทดสอบตำแ...,5.0
13,13,Q1,แฮมทาโร่ควรใช้ a b testing เนื่องจากการใช้ a b...,1.0


# Train Model

In [73]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, ElasticNet, Lasso
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
import pickle  # For saving models
from xgboost import XGBRegressor

In [74]:

# Load dataset
df = df_train

# Dictionary to store models
models = {}

# Train a separate model for each question
for q in df["set"].unique():
    print(f"Training model for {q}...")
    
    # Filter data for the current question
    df_q = df[df["set"] == q]
    
    # Prepare features and labels
    X = df_q["answer"]
    y = df_q["score"]

    # Vectorize text
    vectorizer = TfidfVectorizer()
    X_vectorized = vectorizer.fit_transform(X)
    
    # Random Forest Regressor
    # ========================================
    # model_name = 'RandomForestRegressor'
    # model = RandomForestRegressor(n_estimators=100)
    # model.fit(X_vectorized, y)


    # SVD
    # ========================================
    model_name = 'SVR'
    # Define hyperparameter grid
    param_grid = {
        "C": [0.1, 1, 10, 100],  # Regularization strength
        "epsilon": [0.01, 0.1, 0.5],  # Error margin for regression
        "kernel": ["linear", "rbf"]  # Linear or RBF kernel
    }
    svr = SVR()
    grid_search = GridSearchCV(svr, param_grid, scoring="neg_mean_squared_error", cv=5, n_jobs=-1)
    grid_search.fit(X_vectorized, y)
    best_params = grid_search.best_params_
    print(f"Best hyperparameters: {best_params}")
    
    model = SVR(kernel=best_params["kernel"], C=best_params["C"], epsilon=best_params["epsilon"])
    model.fit(X_vectorized, y)
    
    
    # ElasticNet
    # # ========================================
    # model_name = 'ElasticNet'
    # # Define hyperparameter grid
    # param_grid = {
    # "alpha": [0.01, 0.1, 1, 10],
    # "l1_ratio": [0.1, 0.5, 0.9]  # 0 = Ridge, 1 = Lasso
    # }

    # # Perform GridSearch
    # elastic_net = ElasticNet()
    # grid_search = GridSearchCV(elastic_net, param_grid, scoring="neg_mean_squared_error", cv=5)
    # grid_search.fit(X_vectorized, y)
    
    # model = ElasticNet(alpha=grid_search.best_params_["alpha"], l1_ratio=grid_search.best_params_["l1_ratio"])
    # model.fit(X_vectorized, y)
    
    
    # XGBoost
    # ========================================
    # model_name = 'XGBRegressor'
    # model = XGBRegressor()
    # param_grid = {
    #     "n_estimators": [100, 200, 300],
    #     "max_depth": [3, 4, 5],
    #     "learning_rate": [0.01, 0.1, 0.3]
    # }
    # grid_search = GridSearchCV(model, param_grid, scoring="neg_mean_squared_error", cv=5)
    # grid_search.fit(X_vectorized, y)
    # best_params = grid_search.best_params_
    # print(f"Best hyperparameters: {best_params}")
    
    # model = XGBRegressor(n_estimators=best_params["n_estimators"], max_depth=best_params["max_depth"], learning_rate=best_params["learning_rate"])
    # model.fit(X_vectorized, y)
    
    
    
    # Save model and vectorizer
    with open(f"weights/model_{q}.pkl", "wb") as f:
        pickle.dump((vectorizer, model), f)



Training model for Q2...
Best hyperparameters: {'C': 10, 'epsilon': 0.01, 'kernel': 'linear'}
Training model for Q3...
Best hyperparameters: {'C': 10, 'epsilon': 0.1, 'kernel': 'linear'}
Training model for Q1...
Best hyperparameters: {'C': 10, 'epsilon': 0.01, 'kernel': 'linear'}
Training model for Q4...
Best hyperparameters: {'C': 10, 'epsilon': 0.01, 'kernel': 'linear'}


In [75]:
import pickle

# Load test set
test_df = df_test

# Prepare output list
predictions = []

for _, row in test_df.iterrows():
    question = row["set"]
    answer = row["answer"]

    # Load the correct model for the question
    with open(f"weights/model_{question}.pkl", "rb") as f:
        vectorizer, model = pickle.load(f)

    # Transform answer and predict score
    X_test_vectorized = vectorizer.transform([answer])
    predicted_score = model.predict(X_test_vectorized)[0]
    
    rounded_score = np.clip(round(predicted_score * 4) /4 , 0 , 5)

    predictions.append({"ID": row["ID"], "score": rounded_score})

# Convert to DataFrame and save
pred_df = pd.DataFrame(predictions)
pred_df.to_csv(f"predictions/{model_name}.csv", index=False)


In [ ]:
sample_record = df_train.sample(5)[0]

q = sample_record['set']
a = sample_record['answer']
s = sample_record['score']

In [77]:
predictions = pd.read_csv(f"predictions/{model_name}.csv")

predictions.describe()

,ID,score
count,90.000000,90.000000
mean,406.500000,3.194444
std,26.124701,1.258554
min,362.000000,0.000000
25%,384.250000,2.500000
50%,406.500000,3.250000
75%,428.750000,4.250000
max,451.000000,5.000000
